In [12]:
from langchain.llms  import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.utilities import SQLDatabase

import sys
import pandas as pd
import os
import json
import openai
import time
import ast
import sqlite3

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [2]:
OPENAI_API_KEY = 'ADD_YOUR_API_KEY_HERE'
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

model_prices = [{'model':'gpt-4-1106-preview','input_price':10,'output_price':30},
                {'model':'gpt-3.5-turbo-0125','input_price':0.5,'output_price':1.5},
                {'model':'davinci-002','input_price':2,'output_price':2},
                {'model':'babbage-002','input_price':0.4,'output_price':0.4}]

client = openai.OpenAI()

#change model name to use different model
model_name = "gpt-3.5-turbo-0125"

#For davinci of babage use this generation function
def base_model_generation(prompt):
  response = openai.completions.create(
    model = model_name ,
    prompt=prompt,
    n = 1,
    stream = False,
    temperature=0.0,
    max_tokens=600,
    top_p = 1.0,
      
    frequency_penalty=0.0,
    presence_penalty=0.0,
    stop = ["SQL"]
  ) 
  return response.choices[0].text, response.usage.prompt_tokens, response.usage.completion_tokens

#For GPT3.5 and GPT4 use this generation function
def GPT_generation(system_prompt, user_prompt):
  response = client.chat.completions.create(
    model= model_name ,
    messages=[{"role": "system", "content": system_prompt},{"role": "user", "content": user_prompt}],
    n = 1,
    stream = False,
    temperature=0.0,
    max_tokens=600,
    top_p = 1.0,
    
    frequency_penalty=0.0,
    presence_penalty=0.0,
  )
  return response.choices[0].message.content, response.usage.prompt_tokens, response.usage.completion_tokens



In [13]:
SPIDER_DB_DIR = '/Users/joaquinoldan/Documents/MASTER_AI/4-Tesis/spider/database/' #the path were all databases exist
DATASET_SCHEMA = '/Users/joaquinoldan/Documents/MASTER_AI/4-Tesis/spider/tables.json'
DATASET = '/Users/joaquinoldan/Documents/MASTER_AI/4-Tesis/spider/dev.json'

OUTPUT_FILE = '/Users/joaquinoldan/Documents/MASTER_AI/4-Tesis/A -Github/SQL_GPU/results/gpt'
INPUT_FILE = '/Users/joaquinoldan/Documents/MASTER_AI/4-Tesis/A-Github/lalossSQL/SQL_GPU/results/gpt/tables-GTP3.5-turbo-0-300.json'

def load_data_json(DATASET):
    # Open the JSON file and load data
    with open(DATASET, 'r') as json_file:
        return json.load(json_file)

In [15]:

# Modificar esto eventualmente por una tecnica de RAG
def getSQLDatabase(db_id):
    # Modificar el PATH para donde este la base de datos
    db_path = SPIDER_DB_DIR
    uri = 'sqlite:///' + db_path + db_id + '/' + db_id + '.sqlite'
    db = SQLDatabase.from_uri(uri,  sample_rows_in_table_info=0)
    return db


def string_to_array(input):
    return ast.literal_eval(input)

def correctCase(llm_table_list, db_SQLDatabase):
    corrected_tables = []
    db_table_list = db_SQLDatabase.get_usable_table_names()
    for table_name in llm_table_list:
        for table in db_table_list:
            if table_name.casefold() == table.casefold():
                corrected_tables.append(table)
    return corrected_tables

def getPertinentSchema(llm_table_list, db_SQLDatabase):
    return db_SQLDatabase.get_table_info(llm_table_list)
    

In [16]:
prompt_system = """### Task
        Given the following SQL tables, your job is to write SQLite queries given a user’s request. Respond in one line.
        {schema}

        """

SQLite_System_Prompt = PromptTemplate(
    input_variables=["question"],
    template= prompt_system
)




In [11]:


if __name__ == '__main__':
    CODEX = load_data_json(INPUT_FILE)
    for index, row in enumerate(CODEX):
        print("")
        print(f"index is {index}")
        if not row['error_encountered']:
            db_SQLDatabase = getSQLDatabase(row['db_id'])
            error_encountered = False
            error = None
            try:
                llm_table_list = string_to_array(row['table_list'])
                corrected_llm_table_list = correctCase(llm_table_list, db_SQLDatabase)
            except Exception as e:
                error = e
                error_encountered = True
            if not error_encountered:
                pertinent_schema = getPertinentSchema(corrected_llm_table_list, db_SQLDatabase)
                if pertinent_schema:
                    try:                    
                        system_prompt = SQLite_System_Prompt.format(schema = pertinent_schema)
                        #predicted_query, prompt_tokens, completion_tokens = base_model_generation(system_prompt, row['question'])
                        predicted_query, prompt_tokens, completion_tokens = GPT_generation(system_prompt, row['question'])
                        print(row['gold_query'])
                        print(predicted_query)
                    except Exception as e:
                        print('e')
                        error = e
                        error_encountered = True
            if not error_encountered:
                row.update({'predicted_query': predicted_query, 'prompt_tokens':prompt_tokens, 'completion_tokens':completion_tokens})
            else:
                row['error_encountered'] = True
                row['error_type'] = type(error).__name__
                row['error_message'] = str(error)
            
    OUTPUT_DIR = '/home/llmuser/models/results/'
    OUTPUT_FILE_DIFF = 'test.json'
    OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILE_DIFF)
    # Save the object to a JSON file
    with open(OUTPUT_PATH, 'w') as json_file:
        json.dump(CODEX, json_file)


index is 0
SELECT count(*) FROM singer
SELECT COUNT(*) FROM singer

index is 1
SELECT count(*) FROM singer
SELECT COUNT(*) FROM singer

index is 2
SELECT name ,  country ,  age FROM singer ORDER BY age DESC
SELECT Name, Country, Age FROM singer ORDER BY Age DESC

index is 3
SELECT name ,  country ,  age FROM singer ORDER BY age DESC
SELECT Name, Country, Age FROM singer ORDER BY Age DESC

index is 4
SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
SELECT AVG(Age) AS average_age, MIN(Age) AS min_age, MAX(Age) AS max_age FROM singer WHERE Country = 'France'

index is 5
SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
SELECT AVG(Age) AS average_age, MIN(Age) AS min_age, MAX(Age) AS max_age FROM singer WHERE Country = 'France'

index is 6


KeyboardInterrupt: 

# Evaluation

In [26]:
OUTPUT_DIR = '/home/llmuser/models/results/all-tables-pred-mixtral-sqlcoder-7b-2.json'

def getdbPath(db_id):
    # Modificar el PATH para donde este la base de datos
    db_path = SPIDER_DB_DIR
    db_uri = db_path + db_id + '/' + db_id + '.sqlite'
    return db_uri

def queryDB(database, SQL_query):
    conn = sqlite3.connect(database)
    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    # Create a table
    cursor.execute("PRAGMA case_sensitive_like = OFF")
    cursor.execute(SQL_query)
    results = cursor.fetchall()
    return results
    # Close the cursor and connection
    cursor.close()
    conn.close()

def correctQuery(CODEX):
    for row in CODEX:
        if not row['error_encountered']:
            row['predicted_query'] = row['predicted_query'].replace('ILIKE', 'LIKE', -1)
            row['predicted_query'] = row['predicted_query'].replace('[/SQL]', '', -1)

def evaluatePredict(CODEX):
    count = 0
    for row in CODEX:
        predict_query_valid = True
        if not row['error_encountered']:
            db = getdbPath(row['db_id'])
            error_encountered = False
            error = None
            error_type = None
            try:
                goldQueryValue =  queryDB(db, row['gold_query'])
            except Exception as e:
                error_encountered = True
                error_type = 'Cannot_Query_Gold'
                error_message = str(e)
                predict_query_valid = False
                goldQueryValue = None
            pred_query = row['predicted_query']
            if not error_encountered:
                try:
                    predQueryValue = queryDB(db, pred_query)
                except Exception as e:
                    error_encountered = True
                    error_type = 'Cannot_Query_Pred'
                    error_message = str(e)
                    predQueryValue = None
            if not error_encountered:
                if goldQueryValue == predQueryValue :
                    predict_success = True
                else:
                    predict_success = False
                    error_encountered = True
                    error_type = 'Gold_Predict_Missmatch'
                    error_message = None
            row.update({'predict_success':predict_success, 'gold_result':goldQueryValue, 'predict_result': predQueryValue,'predict_query_valid':predict_query_valid})
            if error_encountered:
                row['error_encountered'] = error_encountered
                row['error_type'] = error_type
                row['error_message'] = error_message
    return CODEX


CODEX = load_data_json(OUTPUT_DIR)

correctQuery(CODEX)
CODEX_OUT = evaluatePredict(CODEX)

In [27]:
OUTPUT_DIR = '/home/llmuser/models/results/'

OUTPUT_FILE_DIFF = 'all-tables-pred-mixtral-sqlcoder-7b-2-final-results.json'

OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILE_DIFF)

# Save the object to a JSON file
with open(OUTPUT_PATH, 'w') as json_file:
    json.dump(CODEX_OUT, json_file)